Day 1 (https://adventofcode.com/2023/day/1)

In [1]:
import re

numbers_dict = {
    '0': '0',
    '1': '1',
    '2': '2',
    '3': '3',
    '4': '4',
    '5': '5',
    '6': '6',
    '7': '7',
    '8': '8',
    '9': '9',
    'zero': '0',
    'one': '1',
    'two': '2',
    'three': '3',
    'four': '4',
    'five': '5',
    'six': '6',
    'seven': '7',
    'eight': '8',
    'nine': '9'
}

# function for pt2 which transforms code with number-words into code with numbers
def transform_code(code):
    output = ['']*len(code)
    for k in numbers_dict:
        for x in re.finditer(k, code):
            output[x.start()] = numbers_dict[k]
    
    return ''.join(filter(lambda x: x, output))

with open('./inputs/day01.txt', 'r') as f:
    # read in the codes
    codes = list(map(lambda x: x.strip(), f.readlines()))

    # keep only the digits
    part1_codes = [''.join(filter(lambda x: x.isdigit(), c)) for c in codes]

    # keep only first and last digist and sum everything to get our answer
    print('Answer to Day 1, Part 1:', sum(map(int, [c[0] + c[-1] for c in part1_codes])))

    # converting word numbers into digits
    part2_codes = list(map(transform_code, codes))

    # keep only first and last digist and sum everything to get our answer
    print('Answer to Day 1, Part 2:', sum(map(int, [c[0] + c[-1] for c in part2_codes])))

Answer to Day 1, Part 1: 54338
Answer to Day 1, Part 2: 53389


Day 2 (https://adventofcode.com/2023/day/2)

In [2]:
import re

with open('./inputs/day02.txt', 'r') as f:
    # read in the games, store as dict
    gamelines = list(map(lambda x: x.strip(), f.readlines()))
    games = {}
    for l in gamelines:
        id_match = re.search('[0-9]+(?=:)', l)
        id = int(id_match.group(0))
        games[id] = [
            {
                'r': int(re.search('[0-9]+(?= red)', x).group(0)) if re.search('[0-9]+(?= red)', x) else 0,
                'g': int(re.search('[0-9]+(?= green)', x).group(0)) if re.search('[0-9]+(?= green)', x) else 0,
                'b': int(re.search('[0-9]+(?= blue)', x).group(0)) if re.search('[0-9]+(?= blue)', x) else 0,
            }
            for x in l.split(':')[1].split(';')
        ]

    # sum up ids that are still possible
    part1_ans = sum((id for id in games if all((d['r'] <= 12 and d['g'] <= 13 and d['b'] <= 14 for d in games[id]))))
    print('Answer to Day 2, Part 1:', part1_ans)

    # sum up product of min number of cubes for each game
    part2_ans = sum((max(d['r'] for d in games[id])*max(d['g'] for d in games[id])*max(d['b'] for d in games[id]) for id in games))
    print('Answer to Day 2, Part 2:', part2_ans)

Answer to Day 2, Part 1: 3035
Answer to Day 2, Part 2: 66027


Day 3 (https://adventofcode.com/2023/day/3)

In [3]:
import numpy as np
import re

with open('./inputs/day03.txt', 'r') as f:
    lines = f.read().split('\n')

    # schematic as numpy array
    a = np.array(list(map(list, lines)))

    # use numpy array to search for symbols around each part number
    part1_ans = 0
    for i, l in enumerate(lines):
        for m in re.finditer('[0-9]+', l):
            search_area = a[max(0, i-1): i+2, max(0, m.start()-1): m.end()+1]
            if any(s in search_area for s in ('#', '$', '%', '&', '*', '+', '-', '/', '=', '@')):
                part1_ans += int(m.group(0))

    print('Answer to Day 3, Part 1:', part1_ans)

    # iterating similarly to part 1, but looking for coordinates of gears
    gear_coords = {}
    for i, l in enumerate(lines):
        for m in re.finditer('[0-9]+', l):
            search_area = a[max(0, i-1): i+2, max(0, m.start()-1): m.end()+1]
            for coords in np.argwhere(search_area == '*'):
                coords[0] += max(0, i-1)
                coords[1] += max(0, m.start()-1)
                if int(m.group(0)) in gear_coords:
                    gear_coords[int(m.group(0))].append(tuple(coords))
                else:
                    gear_coords[int(m.group(0))] = [tuple(coords)]

    # inverting a dict and going crazy with comprehension, but it works
    parts_by_gear = {}
    for part in gear_coords:
        for gear_coord in gear_coords[part]:
            if gear_coord in parts_by_gear:
                parts_by_gear[gear_coord].append(part)
            else:
                parts_by_gear[gear_coord] = [part]

    part2_ans = sum((parts_by_gear[gear_coord][0] * parts_by_gear[gear_coord][1] for gear_coord in parts_by_gear if len(parts_by_gear[gear_coord]) == 2))
    print('Answer to Day 3, Part 2:', part2_ans)

Answer to Day 3, Part 1: 532331
Answer to Day 3, Part 2: 82301120


Day 4 (https://adventofcode.com/2023/day/4)

In [4]:
with open('./inputs/day04.txt', 'r') as f:
    lines = f.read().split('\n')
    nums = [{
        'mine': set(map(int, l.split(':')[-1].split('|')[0].split())),
        'winners': set(map(int, l.split(':')[-1].split('|')[1].split())),
    } for l in lines]

    part1_ans = 0
    for n in nums:
        x = len(n['mine'].intersection(n['winners']))
        part1_ans += 2**(x-1) if x > 0 else 0

    print('Answer to Day 4, Part 1:', part1_ans)

    # calculating how many of each card we'll have!
    cards = [1] * len(nums)
    for i, n in enumerate(nums):
        for j in range(len(n['mine'].intersection(n['winners']))):
            cards[i+j+1] += cards[i]

    print('Answer to Day 4, Part 2:', sum(cards))

Answer to Day 4, Part 1: 20829
Answer to Day 4, Part 2: 12648035
